In [1]:
import pandas as pd
import numpy as np

import yfinance as yf
import pandas_datareader.data as web

# Parameters

In [2]:
OUTFILE = 'equity_data.xlsx'

INCLUDE_INDUSTRY_ETFS = False
INCLUDE_SINGLE_NAMES = True

#RF_SOURCE = 'FRED'
RF_SOURCE = 'FRENCH'
#RF_SOURCE = 'ETF'

t0 = "1990-12-31"
t1 = "2022-04-30"

# Equity Data

In [3]:
info = pd.DataFrame(columns=['Description'])

info.loc['SPY'] = 'SPDR S&P500'

# industry etfs
if INCLUDE_INDUSTRY_ETFS:
    info.loc['XLY'] = 'Consumer Staples'
    info.loc['XLP'] = 'Consumer Discretionary'
    info.loc['XLE'] = 'Energy'
    info.loc['XLF'] = 'Finance'
    info.loc['XLV'] = 'Health Care'
    info.loc['XLI'] = 'Industrial'
    info.loc['XLB'] = 'Materials'
    info.loc['XLK'] = 'Technology'
    info.loc['XLU'] = 'Utilities'

# single-name stocks
if INCLUDE_SINGLE_NAMES:
    # These tickers all have long histories
    info.loc['AAPL'] = 'Apple'
    info.loc['MSFT'] = 'Microsoft'
    info.loc['JPM'] = 'JP Morgan'
    info.loc['JNJ'] = 'Johnson & Johnson'
    info.loc['MMM'] = '3M'
    info.loc['GE'] = 'GE'
    info.loc['DIS'] = 'Discover'
    info.loc['DHR'] = 'Danaher'
    info.loc['HON'] = 'Honeywell'
    info.loc['BAM'] = 'Brookfield'
    info.loc['MCD'] = 'McDonalds'
    info.loc['F'] = 'Ford'
    info.loc['CVX'] = 'Chevron'
    info.loc['CAT'] = 'Caterpillar'
    info.loc['HD'] = 'Home Depot'
    info.loc['LOW'] = 'Lowes'
    info.loc['NUE'] = 'Nucor'
    info.loc['DHI'] = 'DR Horton'
    info.loc['XOM'] = 'Exxon'
    info.loc['BAC'] = 'Bank of America'
    info.loc['FDX'] = 'FedEx'
    info.loc['NYT'] = 'New York Times'
    info.loc['BA'] = 'Boeing'
    info.loc['TM'] = 'Toyota'
    info.loc['WMT'] = 'Walmart'

info

,Description
SPY,SPDR S&P500
AAPL,Apple
MSFT,Microsoft
JPM,JP Morgan
JNJ,Johnson & Johnson
MMM,3M
GE,GE
DIS,Discover
DHR,Danaher
HON,Honeywell


In [4]:
tickers = list(info.index.values)
df = yf.download(tickers, start=t0, end=t1)['Adj Close']

[*********************100%***********************]  26 of 26 completed


In [5]:
prices = df.resample('M').last()
rets = prices.pct_change().iloc[1:,:]
rets.dropna(inplace=True)

# Get Risk-free rate

Use short-term treasury bills.

Pros / cons of various sources.

* Get it from Ken French database via pandas-datareader, but updated with a few months lag.
* Could get it from FRED, but need API-KEY
* Could use ETF like SHV, but only goes back to 2007

In [6]:
if RF_SOURCE=='FRED':
    1+1

elif RF_SOURCE=='FRENCH':
    rawdata = web.DataReader('F-F_Research_Data_Factors', data_source='famafrench',start=t0,end=t1)
    ff = rawdata[0] / 100
    # convert the yyyy-mm to timestamp object. default is to put first of month, but should be end of month
    ff = ff.to_timestamp().resample('M').last()
    rf = ff[['RF']]
    
elif RF_SOURCE=='ETF':
    shv_raw = yf.download(['SHV'], start=t0, end=t1)['Adj Close']
    shv = shv_raw.resample('M').last()
    rf = shv.pct_change().iloc[1:].to_frame()
    rf.columns = ['RF']
    
info.loc['RF'] = RF_SOURCE

In [7]:
rets_all = rets.merge(rf,how='inner',on='Date')
rets = rets_all.drop(columns=['RF'])
retsx = rets_all.sub(rets_all['RF'],axis=0).drop(columns=['RF'])

# Export Data

In [8]:
with pd.ExcelWriter(OUTFILE) as writer:  
    info.to_excel(writer, sheet_name= 'descriptions')
    rets.to_excel(writer, sheet_name='total returns')
    retsx.to_excel(writer, sheet_name='excess returns')